In [18]:
# Main import block
import numpy as np
import pandas as pd
import requests
import cfscrape 
from bs4 import BeautifulSoup
import datetime
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.proxy import Proxy, ProxyType
import re


import time
from tqdm import tqdm

# Upload to google sheets
import gspread
#import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# Upload to yandex
import yadisk

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [19]:
##### Functions to maintain side manipulations #####


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw * 1000
    return int(date_raw)


# Function for uploading dataframes into the google docs
def google_upload(df, sheet_name):
    # Params used to connect to google api
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        "./misc/macro-parser-lme-c2f2972b48fc.json", scope
    )  # security token
    gc = gspread.authorize(credentials)

    # Key params for connection to particular document
    spreadsheet_key = "1WhLiXRcdlkG7NCvHac9unC8ROt4lcbY7GxrOEdezZ9s"  # document id
    wks_name = sheet_name  # sheet name that we use
    df_array = df.to_numpy()
    df_new = pd.DataFrame(df_array, columns=df.columns)
    # df_new.reset_index(drop=True, inplace=True)
    d2g.upload(df_new, spreadsheet_key, wks_name, credentials=credentials)
    # print(f'Uploading to {sheet_name} completed')
    time.sleep(3)


# yandex upload
def yandex_upload():
    token = pd.read_csv("./misc/token.csv", header=None)
    token = token.iloc[0, 0]

    y = yadisk.YaDisk(token=token)

    try:
        y.upload("./data/LME_db.xlsx", "/macro_db/LME_db.xlsx")
    except:
        y.remove("/macro_db/LME_db.xlsx")
        y.upload("./data/LME_db.xlsx", "/macro_db/LME_db.xlsx")


# Session creation via proxy
def get_session(url):

    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            counter += 1
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f"Attempt {counter+1}")

    return response

In [20]:
# Free proxy function
def get_free_proxies():
    url = "https://free-proxy-list.net/"
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    raw_list = []
    proxies = dict()
    trs = soup.find("table").find_all("tr")  # main table

    for i in trs[1:]:
        raw_list.append(i.find_all("td"))  # list of raw data rows

    for i in range(len(raw_list)):  # creating working proxy list
        try:
            if raw_list[i][6].text == "yes":  # taking only https
                proxies[raw_list[i][3].text] = (
                    f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                )
        except IndexError:
            continue

    adress = pd.Series(proxies)  # creating proxy series

    return adress

In [4]:
test = get_free_proxies()

In [12]:
test

Hong Kong        124.156.163.166:3128
Germany              5.9.211.233:8118
Singapore             8.219.97.248:80
Brazil            200.174.198.95:8888
Canada             72.10.160.90:20931
India               152.67.9.179:8100
Japan               20.44.188.17:3129
Mexico            189.240.60.169:9090
United States      34.82.217.181:5555
Netherlands       51.145.176.250:8080
Finland           135.181.226.41:3128
dtype: object

In [11]:
f"{test[0]}"

'124.156.163.166:3128'

In [25]:
################################################################
##########################   LME ###############################
################################################################


def lme_selenium():
    # Получам series proxies
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # Создаем лист с проксямя
    proxy_list = get_free_proxies()

    # Функция на получение дневной инфы
    def get_price(metall=""):
        url = f"https://www.lme.com/en/Metals/Non-ferrous/LME-{metall}#Summary"

        prices = {}

        # Пройдемся по проксям
        for i in range(len(proxy_list)):
            proxy = Proxy(
                {
                    "proxyType": ProxyType.MANUAL,
                    "httpProxy": f"{proxy_list[i]}",
                    "sslProxy": f"{proxy_list[i]}",
                }
            )

            options = webdriver.ChromeOptions()
            options.add_argument(f"--proxy-server=http://{proxy_list[i]}")

            service = Service()

            driver = webdriver.Chrome(service=service, options=options)

            try:
                # driver.maximize_window()
                driver.get(url=url)

                time.sleep(2)

                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )
                driver.execute_script(
                    "window.scrollTo(0, window.scrollY + window.innerHeight);"
                )

                time.sleep(2)

                html_code = driver.page_source

                soup = BeautifulSoup(html_code, "html.parser")
                data_raw = soup.find_all("td", class_="data-set-table__cell")
                price = pd.to_numeric(data_raw[1].text)
                prices[metall] = price
                break

            except Exception as error:
                print(error)
                continue

            finally:
                driver.close()
                driver.quit()
                return prices

    al_price = get_price("Aluminium")
    cu_price = get_price("Copper")
    zn_price = get_price("Zinc")
    nk_price = get_price("Nickel")
    ld_price = get_price("Lead")

    '''data_row = pd.DataFrame(
        {
            "aluminium": al_price,
            "copper": cu_price,
            "lead": ld_price,
            "nickel": nk_price,
            "zink": zn_price,
        },
        index=[0],
    )'''
    
    data_row = pd.DataFrame({
        "aluminium": al_price,
        "copper": cu_price,
        "lead": ld_price,
        "nickel": nk_price,
        "zink": zn_price,
        },index=[0],)
    

    return data_row

In [ ]:
lme_selenium()